In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.widgets import Slider

In [2]:
LABELS = ["Alt_Si3N4", "Anc_Si3N4", "lambda", "n_eff", "A_eff"]

with h5py.File("Datos.h5", "r") as f:
    A = f["/A"][...]           # p.ej. (5, 1, 1, 1)
    print("shape bruto:", A.shape)

# --- normaliza: lleva el eje de tamaño 5 al último ---
axis5 = next((ax for ax, s in enumerate(A.shape) if s == 5), None)
if axis5 is None:
    raise ValueError("No encontré un eje de tamaño 5 en el dataset")
if axis5 != A.ndim - 1:
    A = np.moveaxis(A, axis5, -1)

print("shape normalizado:", A.shape)   # ahora debe ser (n_alt, n_anc, n_lamb, 5)

def print_voxel(arr, i, j, k):
    ii, jj, kk = i-1, j-1, k-1     # Lumerical es 1-based
    vals = arr[ii, jj, kk, :]      # vector de 5
    for name, v in zip(LABELS, vals):
        print(f"{name:10s} = {v:.6g}")

# ejemplo (con tus números actuales seguramente es 1,1,1)
print_voxel(A, 1, 1, 1)

shape bruto: (5, 7, 7, 7)
shape normalizado: (7, 7, 7, 5)
Alt_Si3N4  = 0.4
Anc_Si3N4  = 0.5
lambda     = 0.72
n_eff      = 1.80111
A_eff      = 1.76696e-13


In [3]:
def to_dataframe(arr):
    nA, nB, nC, _ = arr.shape
    rows = []
    for ii in range(nA):
        for jj in range(nB):
            for kk in range(nC):
                rec = {"i": ii+1, "j": jj+1, "k": kk+1}
                for p, name in enumerate(LABELS):
                    rec[name] = arr[ii, jj, kk, p]
                rows.append(rec)
    return pd.DataFrame(rows, columns=["i","j","k"]+LABELS)

df = to_dataframe(A)
print(df)

     i  j  k  Alt_Si3N4  Anc_Si3N4  lambda     n_eff         A_eff
0    1  1  1   0.400000        0.5    0.72  1.801107  1.766958e-13
1    1  1  2   0.433333        0.5    0.72  1.813994  1.852233e-13
2    1  1  3   0.466667        0.5    0.72  1.826820  1.984774e-13
3    1  1  4   0.500000        0.5    0.72  1.841135  2.036435e-13
4    1  1  5   0.533333        0.5    0.72  1.852436  2.087033e-13
..  .. .. ..        ...        ...     ...       ...           ...
338  7  7  3   0.466667        1.0    0.89  1.852366  3.479552e-13
339  7  7  4   0.500000        1.0    0.89  1.863912  3.624442e-13
340  7  7  5   0.533333        1.0    0.89  1.873923  3.772712e-13
341  7  7  6   0.566667        1.0    0.89  1.882529  3.922862e-13
342  7  7  7   0.600000        1.0    0.89  1.890089  4.075868e-13

[343 rows x 8 columns]
